In [ ]:
import sys
sys.path.append("/home/dictus/dmt/v2")
from neuro_dmt.library.composition import CellDensityValidation, INHRatioValidation

In [ ]:
import pandas as pd
example_cell_density_data = pd.DataFrame(
        {'layer': ['L1','L2','L3','L4','L5','L6'],
         "samples": [[100e-10, 110e-10, 90e-10],
                     [120e-10, 130e-10, 140e-10],
                     [110e-10, 45e-10, 60e-10],
                     [13e-10, 13e-10, 13e-10],
                     [100e-10, 130e-10, 160e-10],
                     [231e-10, 214e-10, 121e-10]]})


In [ ]:
from neuro_dmt.models.bluebrain.circuit.adapter import O1Adapter
circuit_config = "/gpfs/bbp.cscs.ch/project/proj68/circuits/O1/20190307/CircuitConfig"
circuit_model = O1Adapter(circuit_config)

In [ ]:
CellDensityValidation(data=example_cell_density_data)(circuit_model)

In [ ]:
INHRatioValidation(by=[{'layer': l} for l in['L1', 'L2', 'L3', 'L4', 'L5', 'L6']])\
                        (circuit_model)

In [ ]:
# TODO: SynapseDensityValidation

In [ ]:
# automatically puts mtype in the title, rather than cluttering x-axis with repeated mentions
CellDensityValidation(by=[{'mtype': 'MC', 'layer': 'L{}'.format(l)} for l in range(1, 7)])\
    (circuit_model)

In [ ]:
circuit_model_2 = O1Adapter("/gpfs/bbp.cscs.ch/project/proj66/circuits/O1/r0/CircuitConfig")
CellDensityValidation(data=example_cell_density_data)(circuit_model, circuit_model_2)

In [ ]:
# TODO: how to define generic 'ByMtypeDensityAnalysis' as a subclass of CellDensityAnalysis
#       more generally: how to define validations where 'by' depends on the output of some adapter method (mtypes)

In [ ]:
try:
    INHRatioValidation()
except AssertionError as E:
    print(E)

In [ ]:
import numpy as np
class ActuallyAnO1AtlasAdapter():
    
    def __init__(self, atlas_dir):
        from voxcell.nexus.voxelbrain import Atlas
        self._atlas = Atlas.open(atlas_dir)
    
    def _mask_for_query(self, query):
        queryacro = "@{}$".format(query.get('layer'))
        return self._atlas.get_region_mask(queryacro)

    def cell_density(self, query):
        mask = self._mask_for_query(query)
        return ((
            self._atlas.load_data("[cell_density]EXC").raw 
            + self._atlas.load_data("[cell_density]INH").raw)[mask.raw])
        

In [ ]:
import yaml
DefilipeDatapath = "/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/DeFelipe2017.yaml"
with open(DefilipeDatapath) as DFDF:
    DFDict = yaml.load(DFDF)


In [ ]:
import pandas as pd
DefilipeCellDensityData = pd.DataFrame({'layer':
                                        ["L1", "L2", "L3", "L4", "L5", "L6"],
                                       'samples': 
                                        [[subj['densities'][l] 
                                          for subj in DFDict['circuits'].values()]
                                         for l in range(6)]})
DefilipeCellDensityData                                                   

In [ ]:
CellDensityValidation(data=DefilipeCellDensityData)(
    ActuallyAnO1AtlasAdapter("/gpfs/bbp.cscs.ch/project/proj64/dissemination/data/O1/mean_O1/"))

In [ ]:
# test that CellDensityValidation's interface raises the right kind of error
class Amethodlessclass:
    pass
CellDensityValidation(data=example_cell_density_data)(Amethodlessclass())
# it doesn't work